In [20]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [79]:
product_type = "chilled"    #frozen chilled ambient 
num_time_periods = 7        #7 days or 365 days
storage_type="standard"     #standard constrained relaxed 
M = 2000

In [80]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]

In [81]:
# constants definitions
if(product_type=="frozen"):
    if(storage_type=="standard"):
        warehouse_volume = 30
    if(storage_type=="constrained"):
        warehouse_volume = 20
    if(storage_type=="relaxed"):
        warehouse_volume = 50
if(product_type=="chilled"):
    if(storage_type=="standard"):
        warehouse_volume = 250
    if(storage_type=="constrained"):
        warehouse_volume = 180
    if(storage_type=="relaxed"):
        warehouse_volume = 300
if(product_type=="ambient"):
    if(storage_type=="standard"):
        warehouse_volume = 700
    if(storage_type=="constrained"):
        warehouse_volume = 500
    if(storage_type=="relaxed"):
        warehouse_volume = 900
buffer_cost = 25
default_max_order = 10000

# use 1000 or 0 
max_order_exceed_multiplier = 0

In [82]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [83]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [84]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [85]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [86]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start=min(forecast_data['DATE']), end=max(forecast_data['DATE']), freq='D')}).astype(str)
# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

In [87]:
# model object
m = gp.Model()

# decision variables
# Xit
orders = m.addVars(items, time_indexes, vtype=gp.GRB.INTEGER, lb=0)
# Yit
ordered_boolean = m.addVars(items, time_indexes, vtype=gp.GRB.BINARY)
# Sit
storage_used = m.addVars(items, time_indexes, vtype=gp.GRB.INTEGER, lb=0)
# Zt
buffer_storage_used = m.addVars(time_indexes, vtype=gp.GRB.INTEGER, lb=0)
# Dit
clearances = m.addVars(items, time_indexes, vtype=gp.GRB.INTEGER, lb=0)
# Qit
item_max_order_exceeded = m.addVars(items, time_indexes, vtype=gp.GRB.INTEGER, lb=0)

# objective function
ordering_cost_per_tu_objective = gp.quicksum(ordering_cost_per_tu[item] * orders[item, t] for item in items for t in time_indexes)
ordering_cost_fixed_objective = gp.quicksum(ordering_cost_fixed[item] * ordered_boolean[item, t] for item in items for t in time_indexes)
buffer_storage_objective = gp.quicksum(buffer_cost * buffer_storage_used[t] for t in time_indexes)
clearance_objective = gp.quicksum(clearing_cost_per_cu[item] * clearances[item, t] for item in items for t in time_indexes)
max_order_objective = gp.quicksum(max_order_exceed_multiplier * item_max_order_exceeded[item, t] for item in items for t in time_indexes)

m.setObjective(ordering_cost_per_tu_objective + ordering_cost_fixed_objective + buffer_storage_objective + clearance_objective + max_order_objective, sense=gp.GRB.MINIMIZE)

# constraints
# demand satisfaction
for item in demand.keys():
    for t in time_indexes:
        if(t==0):
            m.addConstr(
                (orders[item, t] * cu_per_tu[item])
                -
                storage_used[item, t]
                -
                clearances[item, t]
                >=
                demand[item][t]
                
            )
        else:
            m.addConstr(
                (orders[item, t] * cu_per_tu[item])
                +
                storage_used[item, t-1]
                -
                storage_used[item, t]
                -
                clearances[item, t]
                >=
                demand[item][t]
            )

# inventory volume constraint
for t in time_indexes:
    if(t==0):
        m.addConstr(
            gp.quicksum(
                volume_per_cu[item] * 
                (
                    (cu_per_tu[item] * orders[item, t])
                )
                for item in items
            )
            <=
            warehouse_volume
            +
            buffer_storage_used[t]
        )
    else:
        m.addConstr(
            gp.quicksum(
                volume_per_cu[item] *
                (
                    storage_used[item,t-1]
                    +
                    (cu_per_tu[item] * orders[item, t])
                )
                for item in items
            )
            <=
            warehouse_volume
            +
            buffer_storage_used[t]
        )




# min/max constraints (linking too)
for item in demand.keys():
    for t in time_indexes:
        m.addConstr(ordered_boolean[item, t] >= orders[item, t] / M)
        # m.addConstr(ordered_boolean[item, t] <= orders[item, t])
        m.addConstr(
            orders[item, t]
            >=
            minimum_order_quantity_tu[item] * ordered_boolean[item, t]
        )
        if((not math.isnan(maximum_order_quantity_tu[item])) and t%7!=6):
            m.addConstr(orders[item, t] <= (maximum_order_quantity_tu[item] * ordered_boolean[item, t]) + item_max_order_exceeded[item, t])

for item in demand.keys():
    for t in time_indexes:
        life = shelf_life[item]
        if(t >= life):
            m.addConstr(
                clearances[item, t]
                >=
                gp.quicksum(
                    cu_per_tu[item] * orders[item, t1] for t1 in time_indexes[:t-life]
                )
                -
                gp.quicksum(
                    demand[item][t1] for t1 in time_indexes[:t]
                )
                -
                gp.quicksum(
                    clearances[item, t1] for t1 in time_indexes[:t-1]
                )
            )

# sunday constraint
for item in demand.keys():
    for t in time_indexes:
        if(t%7==6):
            m.addConstr(orders[item, t] == 0)
            m.addConstr(ordered_boolean[item, t] == 0)
            m.addConstr(clearances[item, t] == 0)
            m.addConstr(storage_used[item, t] == storage_used[item, t-1])

m.setParam('TimeLimit', 3*60*60)
m.optimize()

#General Variables
print('Ordering',ordering_cost_per_tu_objective.getValue())
print('Fixing ',ordering_cost_fixed_objective.getValue())
print('Clearance ',clearance_objective.getValue())
print('Max Order ',max_order_objective.getValue())
print('Buffer ',buffer_storage_objective.getValue())

Set parameter TimeLimit to value 10800
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 31655 rows, 69606 columns and 111376 nonzeros
Model fingerprint: 0xdfcca0c5
Variable types: 0 continuous, 69606 integer (13920 binary)
Coefficient statistics:
  Matrix range     [4e-05, 8e+02]
  Objective range  [1e-01, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Presolve removed 7753 rows and 42544 columns
Presolve time: 0.31s
Presolved: 23902 rows, 27062 columns, 78293 nonzeros
Variable types: 0 continuous, 27062 integer (8195 binary)

Root relaxation: objective 4.534078e+05, 20513 iterations, 0.43 seconds (0.66 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 453407.794    0 2496          - 453407.794      -  